In [1]:
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer
from datasets import load_dataset, ClassLabel
import torch.optim as optim
# Define paths and hyperparameters
json_file_path = "C:\\Users\\NARENDRA\\Desktop"
model_name_or_path = "meta-llama/Llama-2-7b-hf"  # Pre-trained Llama2 model
batch_size = 16
learning_rate = 2e-5
num_train_epochs = 3

# Load the JSON dataset with appropriate class labels
def prepare_dataset(json_file_path):
    dataset = load_dataset("json", data_files=['/Responses.json'])
    features = dataset["train"].features

    # Define class labels for tags, patterns, and responses (modify if needed)
    features["tag"] = ClassLabel(names=[])  # Replace with actual tag names
    features["patterns"] = ClassLabel(names=[])  # Replace with actual pattern names
    features["response"] = ClassLabel(names=["[RESPONSE]"])

    # Convert features to numerical labels for training
    dataset = dataset.map(
        lambda examples: {
            "input_ids": tokenizer(examples["tag"] + " " + examples["patterns"], return_tensors="pt")["input_ids"].squeeze(0),
            "labels": tokenizer(examples["response"], return_tensors="pt")["input_ids"].squeeze(0)
        }
    )

    return dataset

# Load the model and tokenizer
tokenizer = LlamaTokenizer.from_pretrained(model_name_or_path)
model = LlamaForCausalLM.from_pretrained(model_name_or_path)

# Prepare the training dataset
train_dataset = prepare_dataset(json_file_path)

# Create DataLoaders for efficient batch training
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Define the optimizer
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_train_epochs):
    model.train()
    losses = []

    for batch in train_dataloader:
        input_ids = batch["input_ids"].to(model.device)
        labels = batch["labels"].to(model.device)

        outputs = model(**batch)
        loss = outputs.loss

        # Perform backward pass and update weights
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        losses.append(loss.item())

    print(f"Epoch {epoch+1}/{num_train_epochs} - Train Loss: {sum(losses)/len(losses)}")

# Save the fine-tuned model
model.save_pretrained("your_fine_tuned_llama2_model")

# Function to generate responses based on tags or patterns
def generate_response(tag, patterns, model, tokenizer, max_length=512):
    input_ids = tokenizer(tag + " " + patterns, return_tensors="pt")["input_ids"].to(model.device)
    generated_ids = model.generate(input_ids, max_length=max_length)
    output = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return output

# Example usage (assuming your JSON file has appropriate tags and patterns)
with open(json_file_path, "r") as f:
    data = json.load(f)

tag = data[0]["tag"]  # Access tag from the first entry (modify as needed)
patterns = data[0]["patterns"]  # Access patterns from the first entry (modify as needed)

response = generate_response(tag, patterns, model, tokenizer)
print(f"Response: {response}")

C:\Users\NARENDRA\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model-00001-of-00002.safetensors:   0%|          | 21.0M/9.98G [00:00<?, ?B/s]

KeyboardInterrupt: 